# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [2]:
names_Babelgum = db.companies.find({"name":"Babelgum"}, {"name":1, "_id":0})
list(names_Babelgum)

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [3]:
employees = db.companies.find({"number_of_employees":{"$gt":5000}}).limit(20).sort("number_of_employees",-1)

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [4]:
companies = db.companies.find({"founded_year":{"$gte":2000, "$lte":2005}}, {"name":1, "founded_year":1, "_id":0})

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [5]:
companies = db.companies.find({"ipo.valuation_amount":{"$gte":100000}, "founded_year":{"$lte":2010}}, {"name":1, "ipo":1, "_id":0})

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [6]:
companies = db.companies.find({"number_of_employees":{"$lt":1000}, "founded_year":{"$lt":2005}}).sort("number_of_employees",-1).limit(10)

### 6. All the companies that don't include the `partners` field.

In [7]:
companies = db.companies.find({"partners":{"$exists":True}})#Como todos incluyen el campo, busco que tenga algo dentro

### 7. All the companies that have a null type of value on the `category_code` field.

In [8]:
companies = db.companies.find({"category_code":None})

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [9]:
companies = db.companies.find({"number_of_employees":{"$gte":100, "$lt":1000}},{"name":1, "number_of_employees":1, "_id":0})

### 9. Order all the companies by their IPO price in a descending order.

In [10]:
companies = db.companies.find({"ipo.valuation_amount":{"$exists":True}}).sort("ipo.valuation_amount", -1)

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [11]:
companies = db.companies.find({"number_of_employees":{"$exists":True}}).sort("number_of_employees", -1).limit(10)

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [12]:
companies = db.companies.find({"founded_month":{"$gte":7}}).limit(1000)

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [13]:
companies = db.companies.find({"founded_year":{"$lt":2000}, "acquisition.price_amount":{"$gt":10000000}})

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [14]:
companies = db.companies.find({"acquisition.acquired_year":{"$gt":2010}}, {"name":1, "acquisition":1, "_id":0}).sort("acquisition.price_amount", -1)

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [15]:
companies = db.companies.find({"founded_year":{"$exists":True}},{"name":1, "founded_year":1, "_id":0}).sort("founded_year", -1)

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [16]:
companies = db.companies.find({"founded_day":{"$lte":7}}).sort("acquisition.price_amount", -1).limit(10)

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [17]:
companies = db.companies.find({"category_code":"web", "number_of_employees":{"$gt":4000}}).sort("number_of_employees", 1)

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [18]:
companies = db.companies.find({"acquisition.price_amount":{"$gt":10000000}, "acquisition.price_currency_code":"EUR"})

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [19]:
companies = db.companies.find({"acquisition.acquired_month":{"$lte":6}},{"name":1, "acquisition":1, "_id":0}).limit(10)

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [20]:
companies = db.companies.find({"founded_year":{"$gte":2000, "$lte":2010}, "acquisition.acquired_year":{"$not":{"$gt":2011}}})

### 20. All the companies that have been 'deadpooled' after the third year.

In [21]:
companies = db.companies.find({"$or":[{"$where":"(this.deadpooled_year - this.founded_year)>3"}, {"$and":[{"$where":"this.deadpooled_year>3"},{"$where":"this.deadpooled_year<1000"}]}]})